In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
#warnings.filterwarnings('ignore')

def load_and_prepare_data(file_path):
    df = pd.read_csv(file_path, index_col=0)
    scaler = MinMaxScaler(feature_range=(0, 1))
    sales_data_scaled = scaler.fit_transform(df)
    return sales_data_scaled, scaler, df.index

     # datos para el LSTM
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(data.shape[0]):
        for j in range(data.shape[1] - n_steps):
            X.append(data[i, j:j+n_steps])
            y.append(data[i, j+n_steps])
    return np.array(X), np.array(y)

# modelo LSTM
def create_model(lstm_units=50, learning_rate=0.001):
    model = Sequential([
        LSTM(lstm_units, activation='relu', input_shape=(n_steps, 1), return_sequences=True),
        LSTM(lstm_units, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

def main():
    # Cargar y preparar los datos
    data, scaler, product_indices = load_and_prepare_data('cisco_forecast_data.csv')
    
    global n_steps
    n_steps = 4  # Usaremos 4 trimestres para predecir el siguiente
    
    # Crear secuencias
    X, y = create_sequences(data, n_steps)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    # Definir el modelo para GridSearchCV
    model = KerasRegressor(model=create_model, verbose=0)
    
    # Definir los hiperparámetros a buscar
    param_grid = {
        'model__lstm_units': [30, 50, 70],
        'model__learning_rate': [0.001, 0.01, 0.1],
        'batch_size': [16, 32, 64],
        'epochs': [50, 100, 150]
    }
    
    # Configurar GridSearchCV
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=TimeSeriesSplit(n_splits=3), n_jobs=-1, verbose=2)
    
    # Ejecutar Grid Search
    grid_result = grid.fit(X, y)
    
    # Imprimir los mejores parámetros
    print("Mejores parámetros encontrados: ", grid_result.best_params_)
    print("Mejor puntuación: ", grid_result.best_score_)
    
    # Usar el mejor modelo para hacer predicciones
    best_model = grid_result.best_estimator_.model_
    
    # Hacer predicciones para el próximo trimestre
    predictions = []
    for i in range(data.shape[0]):
        last_sequence = data[i, -n_steps:].reshape(1, n_steps, 1)
        predicted_scaled = best_model.predict(last_sequence)
        predictions.append(predicted_scaled[0, 0])
    
    # Desnormalizar las predicciones
    predictions_array = np.array(predictions).reshape(-1, 1)
    dummy_array = np.zeros((predictions_array.shape[0], data.shape[1]))
    dummy_array[:, -1] = predictions_array.flatten()
    predicted_sales = scaler.inverse_transform(dummy_array)[:, -1]
    
    # predicciones
    for i, sales in enumerate(predicted_sales):
        print(f"Producto {product_indices[i]}: Ventas previstas para el próximo trimestre: {sales:.2f}")

if __name__ == "__main__":
    main()

Fitting 3 folds for each of 81 candidates, totalling 243 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Library/Frameworks/Py

Mejores parámetros encontrados:  {'batch_size': 16, 'epochs': 100, 'model__learning_rate': 0.001, 'model__lstm_units': 30}
Mejor puntuación:  0.8273275833144632
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Producto 0: Ventas previstas para el próximo trimestre: 115375.21
Producto 1: Ventas previstas par